In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn import metrics
import json
import nltk
from nltk.stem.porter import PorterStemmer
# nltk.download('stopwords')
from nltk.corpus import stopwords
import collections
import pickle
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/robertvandevlasakker/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Load the Data

In [4]:
root = "../data/external/"

# Data Pierre
file_name = root + 'Dataset_Pierre.csv'
df_Pierre = pd.read_csv(file_name, header=[0, 1]) 
df_Pierre = df_Pierre.iloc[: , 1:]

# Data Andrei
file_name = root + 'Dataset_Andrei.csv'
df_Andrei = pd.read_csv(file_name)

# Data Palms
file_name = root + 'Dataset_Kissling.txt'
df_Daniel = pd.read_csv(file_name,
                 sep='\t', encoding='Latin-1')
palm_species = df_Daniel[~df_Daniel.isnull().any(axis=1)]['SpecName'].values
df_Daniel.set_index('SpecName', inplace=True)

# # Values Pierre
# print('Pierre')
# print(df_Pierre.columns.get_level_values(0).unique())

# # Values df_Andrei
# print('Andrei')
# print(df_Andrei.columns.unique())

# # Values df_Andrei
# print('Daniel')
# print(df_Daniel.columns.unique())

In [10]:
sw = list(stopwords.words('english'))
sw.append('like')
sw.append('color')
sw.append('colour')
sw.append('a')
sw.append('x')

### Helper Functions

In [7]:
sw = list(stopwords.words('english'))
sw.append('like')
sw.append('color')
sw.append('colour')
sw.append('a')
sw.append('x')

In [9]:
def jaccard_similarity(A, B):
    """Calculates the Jaccard similarity two sets.

    Args:
        A (Set): Set A
        B (Set): Set B

    Returns:
        Integer: 0.00 - 1.00
    """

    # Create sets just in case
    A = set(A)
    B = set(B)
    
    # Get intersection of two sets
    nominator = A.intersection(B)

    # Find union of two sets
    denominator = A.union(B)

    # Take the ratio of sizes
    similarity = len(nominator)/len(denominator)

    return similarity


def similarity(groundtruth, pred):
    """Calculates the normal similarity between two sets.

    Args:
        A (Set): Set A
        B (Set): Set B

    Returns:
        Integer: 0.00 - 1.00
    """

    # Create sets just in case
    groundtruth = set(groundtruth)
    pred = set(pred)

    # Find intersection of two sets
    nominator = groundtruth.intersection(pred)

    # Find union of two sets
    denominator = groundtruth

    # Take the ratio of sizes
    similarity = len(nominator)/(len(denominator))

    return similarity

## Part of Pierre

In [12]:
root = "../data/processed/"
file_name = root + 'Triples_Pierre.txt'

with open(file_name) as f:
   data = json.load(f)

species = list(data.keys())

In [13]:
# Main Traits 
traits_main_Pierre = df_Pierre.columns.get_level_values(0).unique()[1:]

# Init dict
traits_sub_Pierre = {}

# Extract sub traits per main trait
for main_trait in traits_main_Pierre:

    # Slice dataframe
    sub_traits = list(df_Pierre.xs(main_trait, axis=1).columns)
    sub_traits = [item.lower().split() for item in sub_traits]
    sub_traits = list(set([item for sublist in sub_traits for item in sublist if item not in sw]))
    
    # Split main traits
    for main_traits_split in main_trait.split():

        # Remove main from sub
        sub_traits = list(set(sub_traits) - set([main_traits_split]))
        # Append traits to dict
        traits_sub_Pierre[main_traits_split] = sub_traits
        
    # Original main trais (Just in case)
    traits_sub_Pierre[main_trait] = sub_traits

In [25]:
candidate_dict_pierre = collections.defaultdict(list)

for spss in tqdm(species[0:]):

    for idx, lst in enumerate(data[spss]):

        # Flatten list of lists
        flat_list = [item for sublist in lst for item in sublist]
        main_traits = list(traits_sub_Pierre.keys())

        main_trait_match = set(main_traits) & set(flat_list)
        if main_trait_match:

            gt = set(main_trait_match)
            pred = set(flat_list)
            j_sim = jaccard_similarity(gt, pred)

            candidate_dict_pierre[spss].append((j_sim, flat_list))

            for trait_match in main_trait_match:
                sub_traits = traits_sub_Pierre[trait_match]
                sub_trait_match = set(sub_traits) & set(flat_list)
                if sub_trait_match:

                    gt = set(main_trait_match) | set(sub_trait_match)
                    pred = set(flat_list)
                    j_sim = jaccard_similarity(gt, pred)

                    candidate_dict_pierre[spss].append((j_sim, flat_list))

                    # print('SPECIES', spss)
                    # print('MAIN', main_trait_match) 
                    # print('SUB', sub_trait_match)  
                    # print(jaccard_similarity(gt, pred))    
                    # print('SENT', flat_list, '\n') 






100%|██████████| 360/360 [00:00<00:00, 761.48it/s] 


In [36]:
candidate_dict_pierre_sorted = collections.defaultdict(list)

for species in candidate_dict_pierre.keys():
    candidate_dict_pierre_sorted[species] = candidate_dict_pierre[species].sort(reverse=True)

df_Pierre_own = pd.DataFrame.from_dict(candidate_dict_pierre, orient='index')

In [37]:
df_Pierre_own

,0,1,2,3,4,5,6,7,8,9,...,559,560,561,562,563,564,565,566,567,568
Acacia amythethophylla,"(0.5, [species, stem, stem, stem, stem, vs, vs...","(0.5, [species, stem, stem, bark, bark, stem-b...","(0.42857142857142855, [species, plant, plant, ...","(0.4, [species, trunk, trunk, bark, bark, trun...","(0.375, [species, stem, stem, bark, bark, stem...","(0.3333333333333333, [species, stem, stem, ste...","(0.3333333333333333, [species, root, root, roo...","(0.2857142857142857, [species, trunk, trunk, b...","(0.2857142857142857, [species, root, root, bar...","(0.25, [species, stem, stem, stem, stem, peak,...",...,None,None,None,None,None,None,None,None,None,None
Acacia ataxacantha,"(0.8, [species, plant, plant, type, type, plan...","(0.8, [species, plant, plant, type, type, plan...","(0.8, [species, plant, plant, type, type, plan...","(0.6, [species, plant, plant, type, type, plan...","(0.6, [species, plant, plant, type, type, plan...","(0.5555555555555556, [species, plant, plant, t...","(0.5555555555555556, [species, plant, plant, t...","(0.5555555555555556, [species, plant, plant, t...","(0.3333333333333333, [species, plant, plant, t...","(0.3333333333333333, [species, plant, plant, p...",...,None,None,None,None,None,None,None,None,None,None
Acacia dudgeoni,"(0.5, [species, stem, stem, bark, bark, stem b...","(0.3333333333333333, [species, stem, stem, bar...","(0.3333333333333333, [species, plant, plant, p...","(0.3333333333333333, [species, plant, plant, d...","(0.3333333333333333, [species, leaf, leaf, lea...","(0.25, [species, plant, plant, plant, plant, w...","(0.25, [species, plant, plant, plant, plant, t...","(0.2, [species, plant, plant, lifeform, lifefo...","(0.2, [species, plant, plant, extract, extract...","(0.16666666666666666, [species, stem, stem, ba...",...,None,None,None,None,None,None,None,None,None,None
Acacia ehrenbergiana,"(0.4444444444444444, [species, trunk, trunk, t...","(0.3333333333333333, [species, plant, plant, p...","(0.25, [species, plants, plants, plant, plant,...","(0.25, [species, plants, plants, plant, plant,...","(0.25, [species, leaf, leaf, axil, axil, leaf ...","(0.25, [species, fruits, fruits, fruit, fruit,...","(0.25, [species, fruits, fruits, fruit, fruit,...","(0.25, [species, flowers, flowers, flower, flo...","(0.2222222222222222, [species, trunk, trunk, t...","(0.2222222222222222, [species, plant, plant, p...",...,None,None,None,None,None,None,None,None,None,None
Acacia erythrocalyx,"(0.25, [species, plant, plant, specie, specie,...","(0.2, [species, plant, plant, family, family, ...","(0.125, [species, plants, plants, plant, plant...",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zanthoxylum zanthoxyloides,"(0.6666666666666666, [species, plant, plant, p...","(0.5, [species, stem, stem, bark, bark, stem b...","(0.5, [species, stem, stem, bark, bark, stem b...","(0.5, [species, stem, stem, bark, bark, stem b...","(0.5, [species, stem, stem, bark, bark, stem b...","(0.5, [species, roots, roots, root, root, cepa...","(0.5, [species, roots, roots, root, root, be])","(0.5, [species, roots, roots, root, root, be])","(0.5, [species, root, root, bark, bark, root b...","(0.5, [species, root, root, bark, bark, root b...",...,None,None,None,None,None,None,None,None,None,None
Ziziphus abyssinica,"(0.5, [species, roots, roots, root, root, sieb...","(0.5, [species, roots, roots, root, root, powd...","(0.5, [species, roots, roots, root, root, be])","(0.5, [species, root, root, bark, bark, root b...","(0.5, [species, root, root, bark, bark, root b...","(0.5, [species, root, root, bark, bark, root b...","(0.5, [species, root, root, bark, bark, root b...","(0.5, [species, root, root, bark, bark, root b...","(0.5, [species, root, root, bark, bark, root b...","(0.5, [species, root, root, bark, bark, root b...",...,None,None,None,None,None,None,None,None,None,None
Ziziphus mauritiana,"

## Part Andrei

In [39]:
# Values df_Andrei
print('Andrei')
print(df_Andrei.columns.unique())

Andrei
Index(['Species', 'Family', 'Life form', 'Leaf position', 'Leaf composition',
       'Leaf shape', 'Leaf margin', 'Leaf upper side', 'Leaf lower side',
       'Leaf glands', 'Leaf rachis', 'Thorns/spines', 'Stipules',
       'Inflorescence type', 'Sepals / calyx shape', 'Sepals / calyx numer',
       'Petals / corolla shape', 'Petals / corolla number',
       'Petals / corolla colour', 'Stamen shape', 'Stamen number',
       'Fruit type', 'Fruit shape', 'Fruit colour', 'Aril colour',
       'Seed colour'],
      dtype='object')


In [41]:
# Main Traits 
traits_main_Pierre = df_Pierre.columns.get_level_values(0).unique()[1:]

# Init dict
traits_sub_Pierre = {}

# Extract sub traits per main trait
for main_trait in traits_main_Pierre:

    # Slice dataframe
    sub_traits = list(df_Pierre.xs(main_trait, axis=1).columns)
    sub_traits = [item.lower().split() for item in sub_traits]
    sub_traits = list(set([item for sublist in sub_traits for item in sublist if item not in sw]))
    
    # Split main traits
    for main_traits_split in main_trait.split():

        # Remove main from sub
        sub_traits = list(set(sub_traits) - set([main_traits_split]))
        # Append traits to dict
        traits_sub_Pierre[main_traits_split] = sub_traits
        
    # Original main trais (Just in case)
    traits_sub_Pierre[main_trait] = sub_traits

Pierre
Index(['Species', 'plant type', 'phyllotaxis', 'trunk and root', 'latex',
       'crown', 'stem shape', 'bark', 'bark color', 'leaf shape', 'petiole',
       'leaf blade', 'leaf margin', 'leaf base', 'leaf apex', 'vein',
       'tendril', 'spine', 'blade color', 'fruit', 'XXX', 'inflorescences ',
       'sexuality', 'flower color', 'flower shape'],
      dtype='object')


<!-- # # Values df_Andrei
# print('Daniel')
# print(df_Daniel.columns.unique()) -->